In [17]:
from datasets import load_dataset


mmlu_dataset = load_dataset("cais/mmlu", "all")


In [ ]:
mmlu_dataset['test']

DatasetDict({
    test: Dataset({
        features: ['question', 'subject', 'choices', 'answer'],
        num_rows: 14042
    })
    validation: Dataset({
        features: ['question', 'subject', 'choices', 'answer'],
        num_rows: 1531
    })
    dev: Dataset({
        features: ['question', 'subject', 'choices', 'answer'],
        num_rows: 285
    })
    auxiliary_train: Dataset({
        features: ['question', 'subject', 'choices', 'answer'],
        num_rows: 99842
    })
})

In [10]:
mmlu_queries = {}
mmlu_choices = {}
mmlu_answers = {}
mmlu_subjects = {}

for i, data in enumerate(mmlu_dataset['dev']):
    mmlu_queries[i] = data['question']
    mmlu_choices[i] = data['choices']
    mmlu_subjects[i] = data['subjects']
    mmlu_answers[i] = data['choices'][data['answer']]

In [4]:
from beir import util, LoggingHandler
from beir.retrieval import models
from beir.datasets.data_loader import GenericDataLoader
from beir.retrieval.evaluation import EvaluateRetrieval
from beir.retrieval.search.dense import DenseRetrievalExactSearch as DRES
from sentence_transformers import SentenceTransformer

import logging
import pathlib, os

#### Just some code to print debug information to stdout
logging.basicConfig(format='%(asctime)s - %(message)s',
                    datefmt='%Y-%m-%d %H:%M:%S',
                    level=logging.INFO,
                    handlers=[LoggingHandler()])
#### /print debug information to stdout

#### Download scifact.zip dataset and unzip the dataset
dataset = "MSMARCO"
url = f"https://public.ukp.informatik.tu-darmstadt.de/thakur/BEIR/datasets/{dataset}.zip"
# out_dir = os.path.join('/data/richard/taggerv2/test/test6/beir/outputs', "datasets")
data_path = '/data/richard/taggerv2/test/test6/beir/outputs/datasets/msmarco'

#### Provide the data_path where scifact has been downloaded and unzipped
corpus, queries, qrels = GenericDataLoader(data_folder=data_path).load(split="test")

2025-06-01 19:04:10 - Loading Corpus...


  0%|          | 0/8841823 [00:00<?, ?it/s]

2025-06-01 19:04:33 - Loaded 8841823 TEST Documents.
2025-06-01 19:04:34 - Doc Example: {'text': 'The presence of communication amid scientific minds was equally important to the success of the Manhattan Project as scientific intellect was. The only cloud hanging over the impressive achievement of the atomic researchers and engineers is what their success truly meant; hundreds of thousands of innocent lives obliterated.', 'title': ''}
2025-06-01 19:04:34 - Loading Queries...
2025-06-01 19:04:34 - Loaded 43 TEST Queries.
2025-06-01 19:04:34 - Query Example: anthropological definition of environment


In [5]:
type(corpus)

dict

In [6]:
list(corpus.keys())[0], list(corpus.values())[0], 

('0',
 {'text': 'The presence of communication amid scientific minds was equally important to the success of the Manhattan Project as scientific intellect was. The only cloud hanging over the impressive achievement of the atomic researchers and engineers is what their success truly meant; hundreds of thousands of innocent lives obliterated.',
  'title': ''})

In [7]:
real_corpus = [text_dict['text'] for text_dict in list(corpus.values())]
len(real_corpus)

8841823